In [4]:
import numpy as np
import flopy
import flopy.utils.binaryfile as bf
import matplotlib.pyplot as plt

MF2005_EXE='../mf2005'
MT3DMS_EXE='../mt3dms'

INPUT_AQUIFER  = 'aquifer'
MODEL_NAME = 'aquifer'

In [5]:
# aquifer size


delx = 1
dely = 1
delz = 1

lambdax = 25
lambday = 25
lambdaz = 5

nlay = lambdaz*3
nrow = lambday*3
ncol = lambdax*6

ztop = 0.
zbot = np.zeros((nlay, nrow, ncol))
for i in range(nlay):
    zbot[i, :, :] = ztop - delz*(i+1)
    
# flow boundary condition

ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:, :, 0] = -1
ibound[:, :, -1] = -1
    
strt = np.ones((nlay, nrow, ncol), dtype=np.float32)
strt[:, :, 0] = ncol
strt[:, :, -1] = 0

lrcq = {}
lrcq[0] = []
for i in range(lambdax,lambdax*2):
    for j in range(lambday,lambday*2):
        lrcq[0].append([nlay-1, j, i, 1.])

# contamination source

initial=np.zeros((nlay,nrow,ncol))
for i in range(lambdax,lambdax*2):
    for j in range(lambday,lambday*2):
        initial[nlay-1, j, i] = 1

        
tlim=50
times=np.linspace(0, tlim, num=151)

In [6]:
print(nlay, nrow, ncol)

15 75 150


In [7]:
#load SGeMS
sgems=np.loadtxt(INPUT_AQUIFER, skiprows=1, delimiter=',')

#Reshape SGeMS data
aquifer=np.zeros((nlay,nrow,ncol))
aquifer=np.reshape(sgems.T,(nlay,nrow,ncol))
np.save('sgemsaquifer',aquifer)

In [8]:
hk = np.exp(aquifer)


In [9]:
# Aquifer name
modelname = MODEL_NAME
        
# MODFLOW
mf = flopy.modflow.Modflow(modelname, exe_name=MF2005_EXE)
dis = flopy.modflow.ModflowDis(mf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delx, delc=dely, top=ztop, botm=zbot, nper=1, perlen=tlim)
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(mf, hk=hk)
wel = flopy.modflow.ModflowWel(mf, stress_period_data=lrcq)
oc = flopy.modflow.ModflowOc(mf)
pcg = flopy.modflow.ModflowPcg(mf)
lmt = flopy.modflow.ModflowLmt(mf, output_file_header='extended', output_file_format='formatted', output_file_name=modelname+".ftl")
mf.write_input()
success, buff = mf.run_model()


FloPy is using the following  executable to run the model: ../mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: aquifer.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2020/09/22 19:05:25

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2020/09/22 19:05:31
 Elapsed run time:  5.958 Seconds

  Normal termination of simulation
